In [1]:
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [2]:
df = sns.load_dataset('penguins')
dfcopia = df.copy()
dfcopia = dfcopia.dropna(axis=0)
especies = dfcopia['species'].unique()
print(especies)

dfcopia['specie numerica'] = 0

i = 0
for pinguin in dfcopia['species']:
    #print(pinguin)
    if pinguin == 'Adelie':
        dfcopia['specie numerica'].iloc[i] = 1
        i += 1
    elif pinguin == 'Chinstrap':
        dfcopia['specie numerica'].iloc[i] = 2
        i += 1
    elif pinguin == 'Gentoo':
        dfcopia['specie numerica'].iloc[i] = 3
        i += 1
    #print(dfcopia['species'].iloc[i-1],dfcopia['specie numerica'].iloc[i-1])

features_numericos = ["bill_length_mm","bill_depth_mm", "flipper_length_mm", "body_mass_g"]
df_numericos = dfcopia[features_numericos]

['Adelie' 'Chinstrap' 'Gentoo']


c:\venv\ilumpy\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [3]:
sc = StandardScaler()
sc.fit(df_numericos)
X_norm = sc.transform(df_numericos)

In [4]:
FEATURES = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm"]
TARGET = ["specie numerica"]

In [5]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() #Definindo o normalizador
X_norm = scaler.fit_transform(dfcopia[FEATURES].values) #Normalizando

Normalizamos os dados a fim de realizarmos os PCA.

In [6]:
df_norm = pd.DataFrame(X_norm,columns = FEATURES) #Criando o DataFrame a partir com dados normalizados, que é um array de numpy
df_norm = pd.concat([df_norm,pd.Series(dfcopia['specie numerica']).reset_index(drop=True)],axis=1) #Concatenando a coluna 'price' ao DataFrame

In [7]:
import seaborn as sns
from sklearn.model_selection import train_test_split

TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
FEATURES = ["bill_length_mm", "bill_depth_mm", "flipper_length_mm"]
TARGET = ["specie numerica"]

# pegamos nosso dataset modelo
# dividimos em treino e teste
# escolhemos um número de folds (aqui foram 10)
# temos que ver se 10 é muito ou pouco. quanto menos dados utilizamos menos folds.
# ex: 10 folds em 500 dados
# hiperparametro sem validação cruzada -> melhor candidato; ai aplicamos a validação cruzada

indices = df_norm.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df_norm.loc[indices_treino]
df_teste = df_norm.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values.ravel()
X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values.ravel()

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

NUM_ARVORES = 10
NUM_FOLDS = 5
NUM_CPU_CORES = 4

# aqui temos um melhor rmse devido a floresta aleatoria 
# é bom aumentar os cores
# usar hpc para grande volumes de dados

modelo_rf = RandomForestRegressor(
    n_estimators=NUM_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=NUM_CPU_CORES,
)
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [0.9727593  0.95232352 0.98076923 0.97216288 0.95976262]

A média dos scores é de:  0.9675555103096819


In [9]:
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="max_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-1.  -0.9 -0.8 -1.  -1. ]

A média dos scores é de:  -0.9400000000000001


In [10]:
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="completeness_score",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

c:\venv\ilumpy\lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
c:\venv\ilumpy\lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
c:\venv\ilumpy\lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
c:\venv\ilumpy\lib\site-packages\sklearn\metrics\cluster\_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
c:\venv\ilumpy\lib\site-packages\skl

Os scores foram de:  [0.88535906 0.68883095 0.8229081  0.65999522 0.62004587]

A média dos scores é de:  0.7354278417171677


In [15]:
NUM_ARVORES = 100
NUM_FOLDS = 5
NUM_CPU_CORES = 4

modelo_rf = RandomForestRegressor(
    n_estimators=NUM_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=NUM_CPU_CORES,
)
scores = cross_val_score(
    modelo_rf,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-0.1458938  -0.17449928 -0.11298968 -0.14899105 -0.16085576]

A média dos scores é de:  -0.14864591384088238


In [17]:
from sklearn.metrics import mean_squared_error

NUM_ARVORES = 100
NUM_CPU_CORES = 4

modelo_rf = RandomForestRegressor(
    n_estimators=NUM_ARVORES,
    random_state=SEMENTE_ALEATORIA,
    n_jobs=NUM_CPU_CORES,
)
modelo_rf.fit(X_treino, y_treino)

y_previsao = modelo_rf.predict(X_teste)
RMSE = mean_squared_error(y_teste, y_previsao, squared=False)

print(f"O RMSE do modelo floresta aleatória no conjunto de teste foi de {RMSE}.")

O RMSE do modelo floresta aleatória no conjunto de teste foi de 0.4240040232883479.


In [18]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

NUM_VIZINHOS = 3
NUM_FOLDS = 10

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsRegressor(n_neighbors=NUM_VIZINHOS),
)

# normalizar separadamente para cada kfold que utilizaremos
# knn composto normaliza e faz o kneighbors 
# pipeline = vários comandos em um só, é um tipo mais simples de função
# fazemos a media dos scores/folds que fizemos pq ...?
# fazer otimização de parâmetro e utilizar na validação cruzada

scores = cross_val_score(
    modelo_knn_composto,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-0.         -0.0860663  -0.19245009 -0.         -0.12171612 -0.22771002
 -0.1490712  -0.         -0.06085806 -0.13840913]

A média dos scores é de:  -0.09762809207581946


In [19]:
NUM_VIZINHOS = 7
NUM_FOLDS = 10

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsRegressor(n_neighbors=NUM_VIZINHOS),
)

scores = cross_val_score(
    modelo_knn_composto,
    X_treino,
    y_treino,
    cv=NUM_FOLDS,
    scoring="neg_root_mean_squared_error",
)

print("Os scores foram de: ", scores)
print()
print("A média dos scores é de: ", scores.mean())

Os scores foram de:  [-0.02608203 -0.09759001 -0.19166297 -0.02608203 -0.13299276 -0.17496355
 -0.11065667 -0.         -0.06388766 -0.14529932]

A média dos scores é de:  -0.09692169910070665


In [21]:
NUM_VIZINHOS = 7

modelo_knn_composto = make_pipeline(
    MinMaxScaler(),
    KNeighborsRegressor(n_neighbors=NUM_VIZINHOS),
)
modelo_knn_composto.fit(X_treino, y_treino)

y_previsao = modelo_knn_composto.predict(X_teste)
RMSE = mean_squared_error(y_teste, y_previsao, squared=False)

print(f"O RMSE do modelo k-NN no conjunto de teste foi de {RMSE}.")

O RMSE do modelo k-NN no conjunto de teste foi de 0.11227217828476796.
